# Utilisation des réseaux euclidiens dans pour le décodage de code de dimension 1

## Génération d'équations


On va essayer de décoder des équations de la forme : 

\begin{equation}
 y_i +A_ix + B_i \equiv 0 [p]
\end{equation}
    Que l'on peut reformuler en :
\begin{equation}
y_i + C_i y_0 + D_i \equiv 0 [p] , i = 1\dots h-1, 
\end{equation}

avec  $y_0=y_h$, $C_i = -A_iA_{h}^{-1}$ et $D_i = C_iB_h + B_i$

On va d'abord générer les différents $y_i$

In [1]:
prime_size = 160
h = 2
y = []
a = []
b = []
C = []
D = []
p = random_prime(2^prime_size-1, lbound=2^(prime_size-1))
F = GF(p)
x = F(randint(0,p-1))
for i in range(h):
    a.append(F(randint(0,p-1)))
    b.append(F(randint(0,p-1)))
    y.append(-a[i]*x - b[i])

for i in range(h-1):
    C.append(-a[i]*a[h-1].inverse_of_unit())
    D.append(C[i]*b[h-1] + b[i])
print(C,D)

([848861882270818404268861930307693837097905694043], [954581756250552001308943020753177391651254927746])


On va maintenant générer $n = h-1$ équations, les equations s'écrivent de la forme $y_i + C_iy_0 + D_i \equiv 0 [p]$, c'est a dire si on connait certains bits $c_i$ : $(z_i + c_i) + C_i(z_0 + c_0) + D_i \equiv 0 [p] \implies z_i + u_iz_0 + v_i \equiv 0[p]$

In [2]:
v = vector(ZZ,h)
v[0] = 0
sample = []
arranged_sample = []
known_bytes = 90
mask = 0
for i in range(known_bytes):
    mask += 1<<i

c0 = y[h-1].lift()&mask
z = vector(ZZ,h)
z[0] = y[h-1]-c0

for i in range(h-1):
    j = i+1
    sample.append(F(y[i].lift()+(C[i]*y[h-1]).lift() + D[i].lift()))
    ci = y[i].lift()&mask
    zi = y[i].lift()-ci
    z[j] = zi
    v[j] = ci + C[i]*c0 + D[i].lift()
    arranged_sample.append(z[j] + C[i]*z[0] + v[j])
    print(sample[i]==arranged_sample[i])
    


True


In [3]:
A = zero_matrix(ZZ,h)
A[0,0] = -1
for k in range(1,h):
    A[0,k] = C[k-1]
    A[k,k] = p

def Babai_LLL(A, target):
    B = A.LLL()
    G = B.gram_schmidt()[0]
    diff = target
    for i in reversed(range(B.nrows())):
        diff -=  B[i] * ((diff * G[i]) / (G[i] * G[i])).round()
    return target - diff
print(A.str()+ "\n" + str(C) + "\n" + str(p))

[                                               -1  848861882270818404268861930307693837097905694043]
[                                                0 1018641324410030440135673325761499047341034776531]
[848861882270818404268861930307693837097905694043]
1018641324410030440135673325761499047341034776531


In [4]:
w = Babai_LLL(A,v)
print(str((w-v)) + " , " + str(z) + "," + str(y))

(-337375831635767421675701, -61753168813170608287203) , (615938034227598145103997256652343535138603270144, 98857157380769030037454969716117729767757709312),[98857157380769030038537823230948779247353285382, 615938034227598145105143015139400489028516635219]


In [5]:
print(str(w) + "," + str(v))

(-337375831635767421675701, 332864443521160559068350698729113878879796287878),(0, 332864443521160559068350760482282692050404575081)


In [6]:
F(w[1]-v[1]) == y[1]-(y[1].lift()&mask)

False